In [4]:
import os
import pandas as pd
import numpy as np
import tejapi
import matplotlib.pyplot as plt

In [5]:
os.environ['TEJAPI_KEY'] = 'ZlmnASAtE0FXyBsejatM1lC1goOq32'
os.environ['TEJAPI_BASE'] = 'https://api.tej.com.tw'
tej_key = "ZlmnASAtE0FXyBsejatM1lC1goOq32"
api_base = 'https://api.tej.com.tw'
os.environ['TEJAPI_KEY'] = tej_key
os.environ['TEJAPI_BASE'] = api_base

In [6]:
from zipline.sources.TEJ_Api_Data import get_universe

pool = get_universe(start = '2023-01-03', 
                    end = '2023-01-03',  # 選取 2023 年年初已上市的股票
                    mkt_bd_e = 'TSE',  # 已上市之股票
                    stktp_e = 'Common Stock',  # 普通股 
                    main_ind_e = 'General Industry') # general industry 篩掉金融產業
pool

Currently used TEJ API key call quota 1/1000 (0.1%)
Currently used TEJ API key data quota 835/10000000 (0.01%)


['1101',
 '1102',
 '1103',
 '1104',
 '1108',
 '1109',
 '1110',
 '1201',
 '1203',
 '1210',
 '1213',
 '1215',
 '1216',
 '1217',
 '1218',
 '1219',
 '1220',
 '1225',
 '1227',
 '1229',
 '1231',
 '1232',
 '1233',
 '1234',
 '1235',
 '1236',
 '1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1319',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1339',
 '1342',
 '1402',
 '1409',
 '1410',
 '1413',
 '1414',
 '1416',
 '1417',
 '1418',
 '1419',
 '1423',
 '1432',
 '1434',
 '1435',
 '1436',
 '1437',
 '1438',
 '1439',
 '1440',
 '1441',
 '1442',
 '1443',
 '1444',
 '1445',
 '1446',
 '1447',
 '1449',
 '1451',
 '1452',
 '1453',
 '1454',
 '1455',
 '1456',
 '1457',
 '1459',
 '1460',
 '1463',
 '1464',
 '1465',
 '1466',
 '1467',
 '1468',
 '1470',
 '1471',
 '1472',
 '1473',
 '1474',
 '1475',
 '1476',
 '1477',
 '1503',
 '1504',
 '1506',
 '1512',
 '1513',
 '1514',
 '1515',
 '1516',
 '1517',
 '1519',
 '1521',
 '1522',
 '1524',
 '1525',
 '1526',
 

In [7]:
len(pool)

861

In [8]:
#格雷厄姆選股策略

import TejToolAPI

start_time = pd.Timestamp('2021-01-03')
end_time = pd.Timestamp('2023-01-03')  # 將時間字串轉換成時間格式

data = TejToolAPI.get_history_data(start = start_time,
                                   end = end_time,
                                   ticker = pool,
                                   fin_type = 'A', # 為累計資料，舉例來說，Q3累計：1月～9月的資料。
                                   columns = ['主產業別_中文', 'pbr_tej'],
                                   transfer_to_chinese = True)

data

Currently used TEJ API key call quota 69/1000 (6.9%)
Currently used TEJ API key data quota 1222146/10000000 (12.22%)


,股票代碼,日期,主產業別_中文,股價淨值比_TEJ
0,1101,2021-01-04,M1100 水泥工業,1.2943
1,1102,2021-01-04,M1100 水泥工業,1.0147
2,1103,2021-01-04,M1100 水泥工業,0.6676
3,1104,2021-01-04,M1100 水泥工業,0.8650
4,1108,2021-01-04,M1100 水泥工業,1.1091
...,...,...,...,...
414332,9944,2023-01-03,M9900 其他,0.6726
414333,9945,2023-01-03,M9900 其他,6.5469
414334,9946,2023-01-03,M2500 建材營造,0.6179
414335,9955,2023-01-03,M9900 其他,1.4768


In [9]:
latest_list_date = tejapi.fastget('TWN/APISTOCK', coid = pool, paginate = True, chinese_column_name = True, opts = {'columns':['coid', 'mdate', 'list_date']})
data1 = data.merge(latest_list_date, left_on=['股票代碼'], right_on=['公司簡稱'], how='left')
data1.drop(columns=['公司簡稱', '目前狀態'], inplace=True)
data1['已上市天數'] = (data1['日期'] - data1['最近一次上市日']).dt.days
new_onboard_company = data1.query("日期 == 日期.max() & 已上市天數 < 365")['股票代碼'].unique()
new_onboard_company

array(['3715'], dtype=object)

In [10]:
data1 = data1[~data1['股票代碼'].isin(new_onboard_company)].reset_index(drop=True)

In [28]:
#抓取景氣循環股，用以下股票為例
cyclical_keywords = ['鋼鐵', '航運', '煤炭', '石油', '天然氣', '水泥', '紡織', '營建', '汽車']

In [31]:
#pd<2
data_last_1Y = data1[data1['日期'] == '2022-12-30'].reset_index(drop=True)
condition1 = data_last_1Y[data_last_1Y['股價淨值比_TEJ'] < 2] # 篩選符合選股條件的公司

In [34]:
data1['主產業別_中文'] = data1['主產業別_中文'].str.lower()
cyclical_stocks = condition1[condition1['主產業別_中文'].str.contains('|'.join(cyclical_keywords))]
cyclical_stock_codes = set(cyclical_stocks['股票代碼'])
print(cyclical_stock_codes)

{'1460', '1470', '1110', '3004', '2605', '4438', '1468', '1103', '8367', '1108', '1402', '2002', '2008', '2233', '1417', '2610', '2603', '1452', '1521', '1465', '2607', '2069', '5608', '2617', '1449', '2201', '2024', '2038', '2007', '1419', '1474', '4440', '6288', '1418', '4414', '4426', '2012', '1439', '1423', '1467', '2630', '2022', '2204', '1455', '1522', '2606', '2017', '2211', '6605', '1319', '1533', '2615', '2029', '1447', '2611', '2025', '2613', '2032', '1473', '2208', '2247', '1454', '1457', '2618', '1445', '1525', '1409', '2034', '1451', '2010', '1101', '2015', '2027', '1413', '2241', '2228', '2030', '2020', '1440', '2006', '1524', '1459', '1587', '1339', '1475', '2023', '2031', '2033', '1441', '2206', '2608', '1109', '1104', '1463', '2612', '1102', '3346', '2609', '1414', '2636', '1434', '1568', '1444', '1464', '1466'}


In [35]:
len(cyclical_stock_codes)

105